# Installing the Tensorflow Object Detection API

#### Clone the repository and install dependencies.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!git clone -q https://github.com/tensorflow/models.git
!pip install -qU Cython contextlib2 pillow lxml jupyter matplotlib

fatal: destination path 'models' already exists and is not an empty directory.


#### Install the COCO API

In [ ]:
!git clone -q https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../
%cp -r cocoapi/PythonAPI/pycocotools models/research/

fatal: destination path 'cocoapi' already exists and is not an empty directory.
/content/cocoapi/PythonAPI
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
       ^~~
../common/maskApi.c:46:49: note: 

#### Protobuf Compilation and Object Detection API installation

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

! cp object_detection/packages/tf1/setup.py .
! python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1534300 sha256=3d2bb9dba03e6f14d36b62af66fd8e2bf5be0b7f07c903763cd4f2e7b97873e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-v1wc7rte/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
#@title
# !wget -qnc https://github.com/protocolbuffers/protobuf/releases/download/v3.12.3/protoc-3.12.3-linux-x86_64.zip

# !unzip -qq protoc-3.12.3-linux-x86_64.zip

# %cp bin/protoc models/research
# %cd models/research

# import os
# import sys
# directory = 'object_detection/protos'
# protoc_path = './protoc'
# for file in os.listdir(directory):
#     if file.endswith(".proto"):
#         os.system(f"{protoc_path} {directory}/{file} --python_out=.")

# %ls object_detection/protos/

# !python setup.py build
# !python setup.py install

## Set paths

In [ ]:
! export PYTHONPATH=$PYTHONPATH:/content/models/research
! export PYTHONPATH=$PYTHONPATH:/content/models/research/object_detection
! export PYTHONPATH=$PYTHONPATH:/content/models/research/slim

In [ ]:
! python models/research/object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params(False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature(False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_with_example_miner

# Object detection

In [ ]:
!pip install -qU silence-tensorflow

## SETUP

In [ ]:
import silence_tensorflow

import object_detection
import shutil
import os
import tarfile
import six.moves.urllib as urllib
import tensorflow as tf
ROOT_DIR = '/content'

os.chdir(ROOT_DIR)

In [ ]:
WORKSPACE_PATH  = '/content/workspace/'
CHECKPOINT_PATH = '/content/workspace/checkpoint'
OUTPUT_PATH     = '/content/workspace/output'
EXPORTED_PATH   = '/content/workspace/exported'
ANNOTATIONS_PATH = '/content/workspace/annotations'
IMAGES_PATH = '/content/workspace/Images'
CONFIG_PATH = '/content/workspace/configs'

LABEL_MAP_PATH      = os.path.join(WORKSPACE_PATH, 'label_map.pbtxt')
TRAIN_CSV_PATH      = os.path.join(ANNOTATIONS_PATH, 'training.csv')
VALID_CSV_PATH      = os.path.join(ANNOTATIONS_PATH, 'validation.csv')
TRAIN_RECORD_PATH   = os.path.join(ANNOTATIONS_PATH, 'train.record')
VALID_RECORD_PATH   = os.path.join(ANNOTATIONS_PATH, 'val.record')

NUM_TRAIN_STEPS =  300

# ---- MOBILEDET 
# MODEL_TYPE = 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19'
# CONFIG_TYPE = 'ssdlite_mobiledet_cpu_320x320_coco_sync_4x4'

# ---- MOBILENET V3 - LARGE 
# MODEL_TYPE = 'ssd_mobilenet_v3_large_coco_2020_01_14'
# CONFIG_TYPE = 'ssdlite_mobilenet_v3_large_320x320_coco'

# ---- MOBILENET V3 = SMALL 
MODEL_TYPE = 'ssd_mobilenet_v3_small_coco_2020_01_14'
CONFIG_TYPE = 'ssdlite_mobilenet_v3_small_320x320_coco'

# ---- INCEPTIONV2
# MODEL_TYPE = 'ssd_inception_v2_coco_2018_01_28'
# CONFIG_TYPE = 'ssd_inception_v2_coco'

# ---- MOBILENET V2
# MODEL_TYPE = 'ssd_mobilenet_v2_coco_2018_03_29'
# CONFIG_TYPE = 'ssd_mobilenet_v2_coco'


In [ ]:
if os.path.isdir(WORKSPACE_PATH):
    shutil.rmtree(WORKSPACE_PATH)    

os.makedirs(WORKSPACE_PATH)
os.makedirs(OUTPUT_PATH)
os.makedirs(EXPORTED_PATH)

##  Generate tfrecords file

In [ ]:
!pip install -qU googledrivedownloader

In [ ]:
# https://drive.google.com/file/d/1L_CvmHYL4Z8mYmJZznNDGnB9G1hYMv3P/view?usp=sharing
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1L_CvmHYL4Z8mYmJZznNDGnB9G1hYMv3P',
                                    dest_path=f'{WORKSPACE_PATH}/SIH_COLAB.zip',
                                    unzip=True)

os.remove(f"{WORKSPACE_PATH}/SIH_COLAB.zip")

Unzipping...Done.


In [ ]:
! python /content/workspace/generate_tfrecords.py --csv_input={TRAIN_CSV_PATH} --output_path={TRAIN_RECORD_PATH} --img_path={IMAGES_PATH}
! python /content/workspace/generate_tfrecords.py --csv_input={VALID_CSV_PATH} --output_path={VALID_RECORD_PATH} --img_path={IMAGES_PATH}

Successfully created the TFRecords: /content/workspace/annotations/train.record
Successfully created the TFRecords: /content/workspace/annotations/val.record


## Generate a Label Map

In [ ]:
labels = {
        1: 'Bus',
        2: 'Car',
        3: 'speed_sign',
        4: 'Stop_sign',
        5: 'Traffic_light',
        6: 'Traffic_sign'
}

In [ ]:
with open(LABEL_MAP_PATH, 'w') as f:
    # Loop through all of the labels and write each label to the file with an id
    for key, val in labels.items():
        f.write('item {\n')
        f.write(f"\tid: {key}\n")
        f.write(f"\tname: '{val}'\n")
        f.write('}\n')

In [ ]:
with open(LABEL_MAP_PATH) as f:
    print(f.read())

item {
	id: 1
	name: 'Bus'
}
item {
	id: 2
	name: 'Car'
}
item {
	id: 3
	name: 'speed_sign'
}
item {
	id: 4
	name: 'Stop_sign'
}
item {
	id: 5
	name: 'Traffic_light'
}
item {
	id: 6
	name: 'Traffic_sign'
}



## Download Model and update Config

In [ ]:
# MODEL

download_base = 'http://download.tensorflow.org/models/object_detection/'
model = MODEL_TYPE + '.tar.gz'
tmp = '/content/checkpoint.tar.gz'

if not os.path.exists(CHECKPOINT_PATH):
  # Download the checkpoint
  opener = urllib.request.URLopener()
  opener.retrieve(download_base + model, tmp)

  # Extract all the `model.ckpt` files.
  with tarfile.open(tmp) as tar:
    for member in tar.getmembers():
      member.name = os.path.basename(member.name)
      if 'model.ckpt' in member.name:
        tar.extract(member, path=CHECKPOINT_PATH)

  os.remove(tmp)

In [ ]:
# CONFIG

import re
from google.protobuf import text_format
from object_detection.utils import config_util
from object_detection.utils import label_map_util


pipeline_skeleton = f'{CONFIG_PATH}/{CONFIG_TYPE}.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(labels.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 24,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VALID_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

print(num_classes)
configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, WORKSPACE_PATH)

6
INFO:tensorflow:Maybe overwriting model.ssd.num_classes: 6
INFO:tensorflow:Maybe overwriting train_config.batch_size: 24
INFO:tensorflow:Maybe overwriting train_input_path: /content/workspace/annotations/train.record
INFO:tensorflow:Maybe overwriting eval_input_path: /content/workspace/annotations/val.record
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint: /content/workspace/checkpoint/model.ckpt
INFO:tensorflow:Maybe overwriting label_map_path: /content/workspace/label_map.pbtxt
INFO:tensorflow:Writing pipeline config file to /content/workspace/pipeline.config


In [ ]:
# with open(f'{WORKSPACE_PATH}/pipeline.config') as f:
#     print(f.read())

## Training

In [ ]:
! rm -rf {OUTPUT_PATH}
!python -m object_detection.model_main \
    --pipeline_config_path=$WORKSPACE_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --num_eval_steps=2

W0721 18:02:58.249024 139893223167872 model_lib.py:758] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 300
I0721 18:02:58.249376 139893223167872 config_util.py:552] Maybe overwriting train_steps: 300
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0721 18:02:58.249559 139893223167872 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0721 18:02:58.249648 139893223167872 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0721 18:02:58.249740 139893223167872 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0721 18:02:58.249840 139893223167872 model_lib.py:774] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_tpu

## Export graph

In [ ]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

print(f'Using {TRAINED_CHECKPOINT_PREFIX}')

Using /content/workspace/output/model.ckpt-300


## Frozen inference graph

In [ ]:
!rm -rf $EXPORTED_PATH

!python -m object_detection.export_inference_graph \
  --pipeline_config_path=$WORKSPACE_PATH/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=$EXPORTED_PATH

Instructions for updating:
Please use `layer.__call__` method instead.
W0721 18:05:09.358046 140676600416128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:11.154951 140676600416128 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:11.234031 140676600416128 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:11.411270 140676600416128 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

## Frozen tflite graph and conversion

In [ ]:
!rm -rf tflite_graph

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={WORKSPACE_PATH}/pipeline.config \
    --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
    --output_directory=$WORKSPACE_PATH/tflite_graph \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0721 18:05:26.081302 140371960784768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:27.876163 140371960784768 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:27.944872 140371960784768 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0721 18:05:28.017526 140371960784768 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

 The outputs of the quantized model represents four arrays, named:
 * TFLite_Detetion_PostProcess    -> detection_boxes
 * TFLite_Detection_PostProcess:1 -> detection_classes
 * TFLite_Detection_PostProcess:2 -> detection_scores
 * TFLite_Detection_PostProcess:3 -> num_detections

In [ ]:
converter = tf.lite.TFLiteConverter.from_frozen_graph('/content/workspace/tflite_graph/tflite_graph.pb', 
                                                      input_shapes = {'normalized_input_image_tensor':(1,320,320, 3)},
                                                      input_arrays = ['normalized_input_image_tensor'],
                                                      output_arrays = ['TFLite_Detection_PostProcess',
                                                                       'TFLite_Detection_PostProcess:1',
                                                                       'TFLite_Detection_PostProcess:2',
                                                                       'TFLite_Detection_PostProcess:3']) 
converter.allow_custom_ops=True 
# converter.target_ops = [
#                         tf.lite.OpsSet.TFLITE_BUILTINS,
#                         tf.lite.OpsSet.SELECT_TF_OPS,

# ]
converter.optimizations = [
                        #    tf.lite.Optimize.DEFAULT,
                        #    tf.lite.Optimize.OPTIMIZE_FOR_SIZE,
                        #    tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
                           ]
tflite_model = converter.convert() 

tf_model_files = f'/content/workspace/{CONFIG_TYPE}.tflite' 
!rm -rf $tf_model_files
open(tf_model_files,"wb").write(tflite_model)

3991484

# Test the Model with TFLite Interpreter 

In [ ]:
import numpy as np

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]["shape"][1]
width = input_details[0]["shape"][2]

floating_model = input_details[0]["dtype"] == np.float32

input_mean = 127.5
input_std = 127.5

In [ ]:
print(input_details[0]["dtype"])

<class 'numpy.float32'>
